In [24]:
from collections import Counter
import pandas as pd

dataset = pd.read_csv('./dataset/SMSSpamCollection_Split', sep=',')
spam_messages = dataset[(dataset['label'] == 'spam') & ((dataset['split'] == 'train') | (dataset['split'] == 'val'))]['text'].str.cat(sep=' ')
ham_messages = dataset[(dataset['label'] == 'ham') & ((dataset['split'] == 'train') | (dataset['split'] == 'val'))]['text'].str.cat(sep=' ')

def top_frequency(text, top_n=10):
    words = text.split()
    word_count = Counter(words)
    top_words = word_count.most_common(top_n)
    return dict(top_words)

top_spam_words = top_frequency(spam_messages, top_n=10)
top_ham_words = top_frequency(ham_messages, top_n=10)

remove_words=["u","ur","txt"]
def rule_based_classification(text, top_spam_words, top_ham_words):
    words = text.split()
    words = [word for word in words if word.lower() not in remove_words]
    word_count = Counter(words)
    spam_score = 0
    ham_score = 0
    
#     based on frequency
    for word, count in word_count.items():
        if word in top_spam_words:
            spam_score += 1
        if word in top_ham_words:
            ham_score += 1

#     based on length
    if 1 <= len(words) <= 10:
        ham_score += 1
    elif 10 <len(words)< 30:
        spam_score += 1
    elif len(words) >= 30:
        ham_score += 1
        
#    make a decison
    if spam_score>ham_score:
        return "spam"
    else:
        return "ham"

correct_samples = 0
total_samples = len(dataset[dataset['split'] == 'test'])
for index, row in dataset[dataset['split'] == 'test'].iterrows():
    text = row['text']
    label = row['label']
    predicted_label = rule_based_classification(text, top_spam_words, top_ham_words)
    if predicted_label == label:
        correct_samples += 1

test_accuracy = correct_samples / total_samples

print("Test Accuracy (Rule-Based): {:.2%}".format(test_accuracy))



Test Accuracy (Rule-Based): 85.59%
